## A Case Study - Power Consumption Optimization
We can apply the basic ideas used in the linear optimization to a problem of power consumption optimization. In energy management, decreasing the peak power demand is an important task. We can optimize the schedules of appliances (for a household for example) such that maximum power demand is minimized.


#### Objective
The objective definition is rather straight forward. If we let $P_i^t$ denote the power consumption for the appliance $i$ at time $t$, then the total power consumption at any time $t$ is $P^t = \sum_i P_i^t$. We can define the maximum power demand $P_{max} = \max P^t$. Then the objective function is then simply:
$$
\min P_{max}
$$
#### Variables
For this problem we can assume the variables are the power consumption of an appliance at any time t. We will denote this with $P_i^t$. For $N$ appliances and $T$ time steps, there will be $N \times T$ variables. For programming convenience, we will assume that all the appliances are either on or off, consuming either $0$ or $P_i$ amount of power. We can introduce binary integer variables $x_i^t$ to denote the state of the appliance $i$ at time $t$.
#### Constraints
We introduced both $p_i^t$ and $x_i^t$. First type of constraint arises due to their dependency.
$$
P_i^t = P_i x_i^t \hspace{1cm} \forall i, t
$$
Another type of constraint is the total energy consumption for appliances. We want the schedules of the applianes to be optimized, but the total amount of runtime (total amount of energy consumption) should not change.
$$
\sum_t P_i^t = E_i \hspace{1cm} \forall i
$$

In [2]:
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpDot, lpSum
from utilities import print_result, print_power_values

In [3]:
appliances = list(range(1, 4))
time_steps = list(range(0, 10))
p = LpVariable.dict('P', (appliances, time_steps), lowBound=0, cat='Continuous')
x = LpVariable.dict('x', (appliances, time_steps), lowBound=0, upBound=1, cat='Integer')

# Rated power for appliances
power_values = {
    1: 100,
    2: 50,
    3: 75
}

# Number of time steps each appliance is required to run
runtimes = {
    1: 5,
    2: 3,
    3: 4
}

# introduce max power as a new variable
p_max = LpVariable('P_max', lowBound=0, cat='Continuous')

problem = LpProblem('Minimize maximum power demand', LpMinimize)

for i in appliances:
    for t in time_steps:
        # constraint P_i_t = x_i_t * P_i
        problem += p[(i, t)] == x[(i, t)] * power_values[i]
    
    # total runtime constraint for each appliance
    problem += lpSum([x[(i, t)] for t in time_steps]) == runtimes[i]
    
for t in time_steps:
    # for any time step t total power consumption <= p_max (to ensure p_max is indeed max power)
    problem += lpSum([p[(i, t)] for i in appliances]) <= p_max

# objective
problem += p_max

problem.solve()
print_power_values(problem, p, appliances, time_steps)

Time	App1	App2	App3
0	100	0	0
1	0	0	75
2	100	0	0
3	0	50	75
4	0	50	75
5	100	0	0
6	0	0	75
7	100	0	0
8	0	50	0
9	100	0	0


We can introduce one more condition. For some appliances, there might be minimum up time which means you cannot turn them off for a certain amount of time once they turn on. Let's say for appliance $2$, the minimum up time is $3$ time steps. We can write this condition as follows:
$$
\begin{align}
(x_2^{t - 1} = 0 \land x_2^{t} = 1) &\implies (x_2^{t+1} = 1 \land x_2^{t+2} = 1) \\
(1 - x_2^{t-1} = 1 \land x_2^{t} = 1) &\implies (x_2^{t+1} = 1 \land x_2^{t+2} = 1) \\
(1 - x_2^{t-1} + x_2^{t} \geq 2) &\implies (x_2^{t+1} + x_2^{t+2} \geq 2) \\
(x_2^{t} - x_2^{t-1} \geq 1) &\implies (x_2^{t+1} + x_2^{t+2} \geq 2)
\end{align}
$$

Then we can use $p \implies q \equiv \lnot p \lor q$ identity to rewrite the above if condition.
$$
(x_2^{t} - x_2^{t-1} \leq 0) \lor (x_2^{t+1} + x_2^{t+2} \geq 2)
$$

Introducing a new binary variables $w_2^t$ and some large constant $M$.
$$
\begin{align}
x_2^{t} - x_2^{t-1} &\leq 0 + Mw_2^t \\
x_2^{t+1} + x_2^{t+2} &\geq 2 - M(1 - w_2^t)
\end{align}
$$

This condition checks if the appliance was off at time step $t - 1$ and on at time step $t$. We write this condition for all t in range $[1, T - 3]$ where $T$ is total number of time steps. We should also write a similar condition that checks if the device is on at time step $0$. With a similar analysis as above, we would come down to write:
$$
\begin{align}
x_2^{0} &\leq 0 + Mw_2^0 \\
x_2^{1} + x_2^{2} &\geq 2 - M(1 - w_2^0)
\end{align}
$$

In [15]:
appliances = list(range(1, 4))
time_steps = list(range(0, 10))
p = LpVariable.dict('P', (appliances, time_steps), lowBound=0, cat='Continuous')
x = LpVariable.dict('x', (appliances, time_steps), lowBound=0, upBound=1, cat='Integer')

# Rated power for appliances
power_values = {
    1: 100,
    2: 50,
    3: 75
}

# Number of time steps each appliance is required to run
runtimes = {
    1: 5,
    2: 3,
    3: 4
}

# introduce max power as a new variable
p_max = LpVariable('P_max', lowBound=0, cat='Continuous')

problem = LpProblem('Minimize maximum power demand', LpMinimize)

for i in appliances:
    for t in time_steps:
        # constraint P_i_t = x_i_t * P_i
        problem += p[(i, t)] == x[(i, t)] * power_values[i]
    
    # total runtime constraint for each appliance
    problem += lpSum([x[(i, t)] for t in time_steps]) == runtimes[i]
    
for t in time_steps:
    # for any time step t total power consumption <= p_max (to ensure p_max is indeed max power)
    problem += lpSum([p[(i, t)] for i in appliances]) <= p_max

M = 10 * len(time_steps) # some large number
w = LpVariable.dict('w', ([2], time_steps), lowBound=0, upBound=1, cat='Integer')

# minimum up time conditions
for t in time_steps[:-2]:
    if t == time_steps[0]:
        problem += x[(2, t)] <= M * w[(2, t)]
    else:
        problem += x[(2, t)] - x[(2, t - 1)] <= M * w[(2, t)]
    
    problem += x[(2, t + 1)] + x[(2, t +2 )] >= 2 - M * (1 - w[(2, t)])

# objective
problem += p_max

problem.solve()
print_power_values(problem, p, appliances, time_steps)

Time	App1	App2	App3
0	100	0	0
1	100	0	0
2	100	0	0
3	100	0	0
4	100	0	0
5	0	0	75
6	0	0	75
7	0	50	75
8	0	50	0
9	0	50	75


We can generalize the condition for any minimum up time $\tau$ and appliance $i$.
\begin{align}
x_2^{t} - x_2^{t-1} &\leq 0 + Mw_2^t \\
\sum_{j=t+1}^{\tau - 1} x_2^{t+1} + x_2^{t+2} &\geq \tau - 1 - M(1 - w_2^t)
\end{align}

In [20]:
appliances = list(range(1, 4))
time_steps = list(range(0, 10))
p = LpVariable.dict('P', (appliances, time_steps), lowBound=0, cat='Continuous')
x = LpVariable.dict('x', (appliances, time_steps), lowBound=0, upBound=1, cat='Integer')

# Rated power for appliances
power_values = {
    1: 100,
    2: 50,
    3: 75
}

# Number of time steps each appliance is required to run
runtimes = {
    1: 5,
    2: 3,
    3: 4
}

# introduce max power as a new variable
p_max = LpVariable('P_max', lowBound=0, cat='Continuous')

problem = LpProblem('Minimize maximum power demand', LpMinimize)

for i in appliances:
    for t in time_steps:
        # constraint P_i_t = x_i_t * P_i
        problem += p[(i, t)] == x[(i, t)] * power_values[i]
    
    # total runtime constraint for each appliance
    problem += lpSum([x[(i, t)] for t in time_steps]) == runtimes[i]
    
for t in time_steps:
    # for any time step t total power consumption <= p_max (to ensure p_max is indeed max power)
    problem += lpSum([p[(i, t)] for i in appliances]) <= p_max
    
min_up_times = {
    1: 5,
    2: 3,
    3: 4
}

M = 10 * len(time_steps) # some large number
w = LpVariable.dict('w', (appliances, time_steps), lowBound=0, upBound=1, cat='Integer')

for i in appliances:
    tau = min_up_times[i]
    if tau == 1:
        continue
    # minimum up time conditions
    for t in time_steps[:-(tau-1)]:
        if t == time_steps[0]:
            problem += x[(i, t)] <= M * w[(i, t)]
        else:
            problem += x[(i, t)] - x[(i, t - 1)] <= M * w[(i, t)]

        problem += lpSum([x[(i, j)] for j in time_steps[t + 1:t+tau]]) >= tau - 1 - M * (1 - w[(i, t)])

# objective
problem += p_max

problem.solve()
print_power_values(problem, p, appliances, time_steps)

Time	App1	App2	App3
0	0	50	0
1	0	50	75
2	0	50	75
3	0	0	75
4	0	0	75
5	100	0	0
6	100	0	0
7	100	0	0
8	100	0	0
9	100	0	0
